In [1]:
import time
import sounddevice as sd
import threading
import numpy as np 
import numpy as np
from reservoirpy.nodes import Reservoir, Identity
import pygame
import time
import os

pygame 2.6.1 (SDL 2.28.4, Python 3.12.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
MAX_THREADS = 4  # Nombre max de threads actifs
active_threads = []  # Liste des threads en cours
lock = threading.Lock()  # Pour éviter des problèmes de threads concurrents

def non_blocking_wait(delay, start_time):
    """Attend jusqu'à ce que (start_time + delay) soit atteint sans bloquer le CPU"""
    while time.perf_counter() < start_time + delay:
        time.sleep(0.001)  # Évite une boucle active
def normalize(sample):
    sample = sample.astype(np.float32)
    return sample / np.max(np.abs(sample) + 1e-6)  # évite div par zéro

def play_sound(sound):
    try:
        sd.play(sound[0], samplerate=sound[1], blocking=True)
    except Exception as e:
        print(f"Erreur pendant lecture audio : {e}")
    finally:
        with lock:
            if threading.current_thread() in active_threads:
                active_threads.remove(threading.current_thread())

In [3]:

#######################################################DRUMS####################################################

# Init pygame mixer
pygame.mixer.init(frequency=44100, size=-16, channels=2, buffer=512)

# Reservoir config
seed = 12
nb_neurons = 6

reservoir = Reservoir(units=nb_neurons, sr=250.09, lr=2.53, seed=seed)
readout = Identity()
reservoir <<= readout
model = reservoir >> readout

first_step = np.zeros((1, nb_neurons))
sample_played = []
penalty = 1
penalty_length = 4


# Charger les samples avec pygame
sample_paths = [
    r"samples/OneShots (24bit WAV)/Kicks/RK_BT7_Kick_04.wav",
    r"samples\OneShots (24bit WAV)\Percusion\RK_DDG1_Perc_02.wav",
    r"samples\OneShots (24bit WAV)\HiHats\RK_DDG1_Hihat_01.wav",
    r"samples\OneShots (24bit WAV)\Toms\RK_BT6_Tom_11.wav",
    r"samples\OneShots (24bit WAV)\Chords\RK_BT6_Chord_14_Fm9.wav"
]

samples = []
for path in sample_paths:
    if os.path.exists(path):
        sound = pygame.mixer.Sound(path)
        samples.append(sound)
    else:
        print(f"Fichier manquant : {path}")

# Boucle de jeu
for i in range(100):
    output = model.call(first_step)
    output[0, sample_played] -= penalty
    cache = np.argmax(output[0])

    print(f"Sample {i}: {cache}")

    if cache < len(samples):
        samples[cache].play()  

    time.sleep(0.2)

    sample_played.append(cache)
    if len(sample_played) > penalty_length:
        sample_played.pop(0)

pygame.mixer.quit()


Sample 0: 0
Sample 1: 2
Sample 2: 1
Sample 3: 2
Sample 4: 4
Sample 5: 2
Sample 6: 0
Sample 7: 2
Sample 8: 1


KeyboardInterrupt: 

In [48]:

#######################################################CHORDS####################################################
# Init pygame mixer
pygame.mixer.init(frequency=44100, size=-16, channels=2, buffer=512)

# Reservoir config
seed = 12
nb_neurons = 6

reservoir = Reservoir(units=nb_neurons, sr=250.09, lr=200.053, seed=seed)
readout = Identity()
reservoir <<= readout
model = reservoir >> readout

first_step = np.zeros((1, nb_neurons))
sample_played = []
penalty = 1
penalty_length = 4


# Charger les samples avec pygame
sample_paths = [
    r"samples\OneShots (24bit WAV)\Chords\RK_BT7_Chord_08_Gm.wav",
    r"samples\OneShots (24bit WAV)\Chords\RK_BT7_Chord_10_Gm.wav",
    r"samples\OneShots (24bit WAV)\Chords\RK_BT7_Chord_10_Gm.wav",
    r"samples\OneShots (24bit WAV)\HiHats\RK_BTG2_Hihat_12.wav",
    r"samples\OneShots (24bit WAV)\Chords\RK_BT7_Chord_12_Gm.wav"
]

samples = []
for path in sample_paths:
    if os.path.exists(path):
        sound = pygame.mixer.Sound(path)
        samples.append(sound)
    else:
        print(f"Fichier manquant : {path}")

# Boucle de jeu
for i in range(100):
    output = model.call(first_step)
    output[0, sample_played] -= penalty
    cache = np.argmax(output[0])

    print(f"Sample {i}: {cache}")

    if cache < len(samples):
        samples[cache].play()  

    time.sleep(0.4)

    sample_played.append(cache)
    if len(sample_played) > penalty_length:
        sample_played.pop(0)

pygame.mixer.quit()


Sample 0: 0
Sample 1: 3
Sample 2: 1
Sample 3: 3
Sample 4: 4


KeyboardInterrupt: 

In [50]:

#######################################################CANARY####################################################
# Init pygame mixer
pygame.mixer.init(frequency=44100, size=-16, channels=2, buffer=512)

# Reservoir config
seed = 12
nb_neurons = 6

reservoir = Reservoir(units=nb_neurons, sr=20.09, lr=20.053, seed=seed)
readout = Identity()
reservoir <<= readout
model = reservoir >> readout

first_step = np.zeros((1, nb_neurons))
sample_played = []
penalty = 1
penalty_length = 4


# Charger les samples avec pygame
sample_paths = [
    r"repertoire\S1-317035-394254-023-rouge6_July_01_2014_33461156.wav",
    r"repertoire\A2-178253-225440-007-rouge6_July_01_2014_53035723.wav",
    r"repertoire\B-242374-303894-050-rouge6_July_01_2014_42083178.wav",
    r"samples\OneShots (24bit WAV)\HiHats\RK_BTG2_Hihat_12.wav",
    r"repertoire\E1-648667-676935-043-rouge6_July_01_2014_41919004.wav"
]

samples = []
for path in sample_paths:
    if os.path.exists(path):
        sound = pygame.mixer.Sound(path)
        samples.append(sound)
    else:
        print(f"Fichier manquant : {path}")

# Boucle de jeu
for i in range(100):
    output = model.call(first_step)
    output[0, sample_played] -= penalty
    cache = np.argmax(output[0])

    print(f"Sample {i}: {cache}")

    if cache < len(samples):
        samples[cache].play()  

    time.sleep(0.4)

    sample_played.append(cache)
    if len(sample_played) > penalty_length:
        sample_played.pop(0)

pygame.mixer.quit()


Sample 0: 0
Sample 1: 3
Sample 2: 1
Sample 3: 3
Sample 4: 4
Sample 5: 3
Sample 6: 0
Sample 7: 3
Sample 8: 1
Sample 9: 3
Sample 10: 4
Sample 11: 3
Sample 12: 0
Sample 13: 3
Sample 14: 1
Sample 15: 3
Sample 16: 4
Sample 17: 3
Sample 18: 0
Sample 19: 3
Sample 20: 1
Sample 21: 3
Sample 22: 4
Sample 23: 3
Sample 24: 0
Sample 25: 3
Sample 26: 1
Sample 27: 3
Sample 28: 4
Sample 29: 3
Sample 30: 0
Sample 31: 3
Sample 32: 1
Sample 33: 3
Sample 34: 4
Sample 35: 3
Sample 36: 0
Sample 37: 3
Sample 38: 1


KeyboardInterrupt: 